In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
from preprocess import clean_police_dataset
import shapely
from shapely.geometry import point

# Anita Sun

In [17]:
def tract_merger(pol_filepath, shapefile_path):
    ''' 
    takes in the police homicide filepath and the census tract shapefile path
    and merges them together for that particular state

    pol_filepath: (str) file path for the police homicide csv
    shapefile_path: (str) file path for the census tract polygons shapefile

    returns:
    geo_joined_df: (GeoPandas Dataframe) a geopandas dataframe
    '''
    gdf = gpd.read_file(shapefile_path)
    pol_df = clean_police_dataset(pol_filepath)
    pol_df['geometry'] = gpd.points_from_xy(pol_df['longitude'], pol_df['latitude'])
    pol_gdf = gpd.GeoDataFrame(pol_df, geometry='geometry', crs="EPSG:4326")
    geo_joined_df = gpd.sjoin(gdf.to_crs(crs='EPSG:26916'), pol_gdf.to_crs(crs='EPSG:26916'))
    geo_joined_df = geo_joined_df[['GEOIDFQ','name', 'age', 'gender', 'race',
       'date', 'street_address', 'city', 'state', 'zip', 'county',
       'agency_responsible', 'ori', 'cause_of_death', 'circumstances',
       'disposition_official', 'officer_charged', 'news_urls',
       'signs_of_mental_illness', 'allegedly_armed', 'wapo_armed',
       'wapo_threat_level', 'wapo_flee', 'geography', 'encounter_type',
       'initial_reason', 'call_for_service', 'tract',
       'hhincome_median_census_tract', 'latitude', 'longitude',
       'pop_total_census_tract', 'pop_white_census_tract',
       'pop_black_census_tract', 'pop_native_american_census_tract',
       'pop_asian_census_tract', 'pop_pacific_islander_census_tract',
       'pop_other_multiple_census_tract', 'pop_hispanic_census_tract',
       'lat_long', 'month', 'day', 'year']]
    # renaming GEOIDFQ to GEO_ID so we can merge with ACS later
    geo_joined_df.rename(columns={'GEOIDFQ':'GEO_ID'}, inplace=True)
    # correcting year column to dtype int64
    geo_joined_df['year'] = geo_joined_df['year'].astype(int)
    # correcting zip column to dtype int64    

    return geo_joined_df

In [15]:
def booleaner(obj, list_like):
    ''' 
    returns 1 if obj is found in list_like, else returns zero
    obj: (anything)
    list_like: (listlike)
    '''
    if obj in list_like:
        return 1
    else:
        return 0

In [13]:
def attr_merger(geo_joined_df, census_file_path, county):
    ''' 
    takes a geo_df, census_file_path, and year and merges them all together
    geo_joined_df: (GeoPandasDataFrame) a sjoin of a state's .shp shapefile and the original police homicide csv
        has column 'GEOIDFQ' renamed to 'GEO_ID' for merging;
    census_file_path: file path for census tract demographic information, should be csv, year = 2013
    county: (str) county in question
    
    returns:
        df: a pd DataFrame containing census demographic data and a column that contains boolean val for pol homicide
    '''
    df = pd.read_csv(census_file_path)
    # dropping row index zero because it contains column name information and not data
    df = df.drop(index=0, axis=0)

    # converting all possible columns to float for later regression/classification analysis
    for col in df.columns:
        try:
            df[col] = df[col].astype(float)
        except:
            continue
        
    df['in_county'] = df.NAME.str.extract(r'\s*([\w\s]+ County)', expand=False)
    
    if type(county)==str:
        df = df[df['in_county'] == county]
    if type(county)==list:
        df = df[df['in_county'].isin(county)]        
    df['target'] = df.apply(
		lambda x: booleaner(x['GEO_ID'], geo_joined_df['GEO_ID'].values),
        axis=1
	)

    # performing merge on the 'GEO_ID' column with the sjoined tract polygon/homicide df
    return df

In [19]:
geo_ny = tract_merger('police_data.csv', 'new_york_shapefiles/tl_2023_36_tract.shp')
geo_il = tract_merger('police_data.csv', 'illinois_shapefiles/tl_2023_17_tract.shp')
geo_ca = tract_merger('police_data.csv', 'california_shapefiles/tl_2023_06_tract.shp')

In [20]:
df_ny = attr_merger(geo_ny, 'census_2013/ny_2013.csv', county=['Bronx County', 'Kings County', 'New York County', 'Queens County', 'Richmond County'])
df_il = attr_merger(geo_il, 'census_2013/il_2013.csv', county='Cook County')
df_ca = attr_merger(geo_ca, 'census_2013/ca_2013.csv', county='Los Angeles County')

/tmp/ipykernel_292988/1461020114.py:12: DtypeWarning: Columns (2,3,58,59,82,83,124,125,126,127,128,129,154,155,156,157,158,159,214,215,216,217,218,219,232,233,234,235,236,237,250,251,252,253,254,255,268,269,270,271,272,273,354,355,356,358,359,362,363,366,367,370,371,374,375,378,379,382,383,386,387,390,391,394,395,398,399,402,403,406,407,410,411,414,415,422,423,426,427,430,431,434,435,438,439,440,442,443,446,447,450,451,452,454,455,458,459,462,463,464,466,467,470,471,474,475,478,479,482,483,486,487,490,491,494,495,498,499,502,503,506,507,510,511,514,515,518,519,522,523,526,527,530,531,534,535,538,539,542,543,546,547,550,551,554,555,558,559,562,563,566,567,570,571,574,575,578,579,582,583,584,586,587,590,591,594,595,598,599,602,603,606,607,610,611,612,614,615,618,619,622,623,626,627,630,631,634,635,638,639,642,643,646,647,650,651,654,655,658,659,662,663,666,667,670,671,674,675) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(census_file_path)
/

In [26]:
df_ny.to_csv('final_merge_ny.csv')
df_ca.to_csv('final_merge_ca.csv')
df_il.to_csv('final_merge_il.csv')